Download text file. It's the first chuck of the Gigawords dataset (https://catalog.ldc.upenn.edu/LDC2003T05)

In [0]:
!wget https://www.dropbox.com/s/b6mmt7w115onvfp/text.gz && gzip -d text.gz

sample of the text

In [0]:
!head text

# Word2vec Demo

In [2]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
from collections import Counter
import numpy as np
np.random.seed(0)

from tqdm import tqdm
from gensim.models import Word2Vec
from typing import List

# workaround a bug
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [0]:
# load the text file
sents = []
with open('text') as fp:
    for line in fp:
        sents.append([word.lower() for word in line.strip().split()])

Train a CBOW model with embeddings size 50, window size 5. We ignore any word with count smaller than 50.

In [0]:
model = Word2Vec(sents, size=50, window=5, min_count=50, sg=0)

In [0]:
class Embeddings:
    def __init__(self, embeddings: np.ndarray, vocabulary: List[str]):
        assert len(embeddings) == len(vocabulary)
        self.embeddings = embeddings
        self.word2id = {w:i for i, w in enumerate(vocabulary)}
        self.id2word = vocabulary

    def vector(self, word: str):
        assert word in self.word2id, f'{word} is not in the vocabulary'
        return self.embeddings[self.word2id[word]]

    def nearest_neighbor(self, query: np.ndarray, topk=10):
        '''
        given a query, print the topk nearest words vectors measured by cosine similarity
        cos_sim(v1, v2) = dot_product(v1, v2) / (||v1|| * ||v2||)
        '''
        ############
        ### TODO ###
        ############
        raise NotImplementedError

    def analogy(self, w1: str, w2: str, w3: str, topk=10):
        v1 = self.vector(w1)
        v2 = self.vector(w2)
        v3 = self.vector(w3)
        query = v1 - v2 + v3
        self.nearest_neighbor(query, topk=topk)

In [0]:
official_embedding = Embeddings(model.wv.vectors, model.wv.index2word)

Find the most similar word under cosine similarity.



In [0]:
word = 'obama' #@param {type: "string"}
word_embedding = official_embedding.vector(word)
official_embedding.nearest_neighbor(word_embedding, topk=5)

Find the word $x$ such that
$$v_{w_1} - v_{w_2} \approx v_{x} - v_{w_3}$$
$$v_{x} \approx  v_{w_1} - v_{w_2} + v_{w_3}$$

In [0]:
w1 = 'king' #@param {type: "string"}
w2 = 'men' #@param {type: "string"}
w3 = 'women' #@param {type: "string"}
official_embedding.analogy(w1, w2, w3, topk=5)

We can also visualize the whole embeddings space using TensorBoard.

In [0]:
writer = SummaryWriter('runs/offical')
writer.add_embedding(model.wv.vectors, model.wv.index2word)
writer.close()

Click the menu -> projector to find the embeddings visualization

In [0]:
%load_ext tensorboard
%tensorboard --logdir runs/offical

# Word2vec from scratch

Hyperparameters


In [0]:
# only keep word with count no less than MIN_COUNT
MIN_COUNT = 50      # @param {type: "number"}
# only keep DATA_SIZE number of examples
DATA_SIZE = 1000000 # @param {type: "number"}
# maximum distance between center word and words in the window
WINDOW_SIZE = 5     # @param {type: "number"}

## Setup the data

In [0]:
UNK = '<UNK>'
UNK_ID = 0

class Vocabulary:
    def __init__(self, filepath, min_count=3):
        vocab = Counter()
        with open(filepath) as fp:
            for line in fp:
                words = [word.lower() for word in line.strip().split()]
                vocab.update(words)
        self.word2id = {UNK: UNK_ID}
        self.id2word = [UNK]
        for word, count in vocab.most_common():
            if count >= min_count:
                self.word2id[word] = len(self.word2id)
                self.id2word.append(word)
        print(f'vocabulary size = {self.vocab_size()}')

    def index_text(self, filepath):
        assert self.word2id is not None
        assert self.id2word is not None
        text = []
        with open(filepath) as fp:
            for line in fp:
                for word in line.strip().split():
                    word = word.lower()
                    if word not in self.word2id:
                        word = UNK
                    text.append(self.word2id[word])
        text = np.array(text)
        print(f'OOV ratio = {sum(text == 0) / len(text) * 100:.2f}%')
        return text

    def vocab_size(self):
        return len(self.word2id)

In [0]:
def build_dataset(text, window_size=5, data_size=None):
    '''
    window_size: context of one size
    '''
    x, y = [], []
    for i in range(window_size,len(text) - window_size):
        left_window = text[i-window_size:i]
        right_window = text[i+1:i+1+window_size]
        window = np.concatenate((left_window, right_window))
        # ignore anything with UNK
        if text[i] == UNK_ID:
            continue
        if (window == UNK_ID).sum() > 2:
            continue
        x.append(window)
        y.append(text[i])
    x, y = np.stack(x), np.stack(y)

    # shuffle data
    idx = np.arange(len(x))
    np.random.shuffle(idx)
    x, y = x[idx], y[idx]

    if data_size is not None:
        x, y = x[:data_size], y[:data_size]

    # split data
    train_size = int(len(x) * 0.9)
    train_x, train_y = x[:train_size], y[:train_size]
    dev_x, dev_y     = x[train_size:], y[train_size:]
    return (train_x, train_y), (dev_x, dev_y)


In [0]:
vocab = Vocabulary('text', min_count=MIN_COUNT)
text = vocab.index_text('text')
print(f'number of tokens = {len(text)}')

In [0]:
(train_x, train_y), (dev_x, dev_y) = build_dataset(text, window_size=WINDOW_SIZE, data_size=DATA_SIZE)
print('training size')
print(train_x.shape, train_y.shape)
print('dev size')
print(dev_x.shape, dev_y.shape)

## Main Model: Continous Bag-Of-Words (CBOW)

Initialize embeddings matrix and each row represents embeddings of a word, let the input embeddings as $V$ and output embeddings as $U$. Output embeddings is used for softmax.

In [0]:
EMBED_SIZE = 50 # 20

input_embed = np.random.randn(vocab.vocab_size(), EMBED_SIZE) * EMBED_SIZE ** -0.5
output_embed = np.random.randn(vocab.vocab_size(), EMBED_SIZE) * EMBED_SIZE ** -0.5

### CBOW forward pass

Assuming we have a slice of words in a sentence $w_{-2}$, $w_{-1}$, $w_{c}$, $w_{1}$, $w_{2}$. We want to predict $w_{c}$ with $w_{-2}$, $w_{-1}$, $w_{1}$, $w_{2}$. Let the vector of a word $w$ from embeddings $V$ and $U$ be $v_w$ and $u_w$, respectively. Let the vocabulary be $\mathcal{V}$.

$$v_{bow} = v_{w_{-2}} + v_{w_{-1}}+ v_{w_{1}}+ v_{w_{2}}$$
$$p(w_c|v_{bow}) = \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}$$
$$\mathcal{L}_{w_c} = - \log p(w_c|v_{bow})$$

In practise, we take the average over $\mathcal{L}_{w_c}$ of a batch of example, as loss of the batch.

### CBOW backward pass

For each sample of $w_{-2}$, $w_{-1}$, $w_{c}$, $w_{1}$, $w_{2}$, we need the work out the gradient of $- \log p(w_c|v_{bow})$ with respect to $u_{w_c}$, $u_w$ where $w \neq w_c$ and $v$ where $v \in \{v_{w_{-2}}, v_{w_{-1}}, v_{w_{1}}, v_{w_{2}}\}$.

Try to work it out

$$\frac{\partial}{\partial u_{w_c}} - \log p(w_c|v_{bow}) = \frac{\partial}{\partial u_{w_c}} - \log \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}$$

$$\frac{\partial}{\partial u_w} - \log p(w_c|v_{bow}) = \frac{\partial}{\partial u_w} - \log \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}$$

$$ \frac{\partial}{\partial v} - \log p(w_c|v_{bow}) = \frac{\partial}{\partial v} - \log \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}$$

Hint 1: 

$\frac{\partial}{\partial v} u \cdot v = u$

$\frac{\partial}{\partial u} u \cdot v = v$

$\frac{\partial}{\partial x} \exp f(x) = \exp \frac{\partial}{\partial x} f(x)$

$\frac{\partial}{\partial x} \log f(x) = \frac {1} {f(x)} \frac{\partial}{\partial x} f(x)$


Hint 2: Additionally, we need to consider the average part when computing the gradient


Hint 3: Let $X = \{x_1, \cdots, x_N\}$, when $x_i$ is to small, it could cause numerical instability when taking log. As a result, we can use the following tricks to compute sotfmax and log jointly. Let $b = \max(X)$,

\begin{align}
\log p(x_i|X)
&= \log \frac {\exp(x_i)} {\sum_j \exp(x_j)} \\
&= \log \frac {\exp(x_i)/\exp(b)} {\sum_j \exp(x_j)/\exp(b)} \\
&= \log \frac {\exp(x_i-b)} {\sum_j \exp(x_j-b)} \\
&= \log \exp(x_i-b) - \log\sum^N_1 \exp(x_j-b)\\
&= x_i - (b + \log\sum^N_1 \exp(x_j - b))\\
\end{align}


In [0]:
# learning rate
LR = 1      # @param {type: "number"}

# batch size
BS = 50     # @param {type: "number"}

step = 0

In [0]:
def forward_and_backward(batch_x, batch_y, input_embed, output_embed):
    # input_embed_grad: [vocab_size, embedding_size]
    input_embed_grad = np.zeros_like(input_embed)
    # output_embed_grad: [vocab_size, embedding_size]
    output_embed_grad = np.zeros_like(output_embed)
    # loss: float
    loss = 0

    ############
    ### TODO ###
    ############

    raise NotImplementedError

    ### Forward Pass
    # window_embed: [BS, window_size *2, embedding_size]
    # bow_embed:    [BS, embedding_size]
    # logits:       [BS, vocab_size]
    # logprobs:     [BS, vocab_size]

    ### Backward Pass
    # probs:             [BS, vocab_size]
    # output_embed_grad: [vocab_size, embedding_size]
    # bow_embed_grad:    [BS, embedding_size]
    # window_embed_grad: [BS, window_size *2, embedding_size]
    # batch_x:           [BS, window_size *2]
    # input_embed_grad:  [vocab_size, embedding_size]

    return loss, input_embed_grad, output_embed_grad

Make sure we get the gradient right

In [0]:
import torch
import torch.nn.functional as F

# use a prime number for batch size would make debugging easier
bs = 7
batch_x = train_x[:bs]
batch_y = train_y[:bs]
loss, input_embed_grad, output_embed_grad = forward_and_backward(batch_x, batch_y, input_embed, output_embed)

_input_embed = torch.tensor(input_embed, requires_grad=True)
_output_embed = torch.tensor(output_embed, requires_grad=True)
_batch_x = torch.tensor(batch_x, requires_grad=False)
_batch_y = torch.tensor(batch_y, requires_grad=False)
_window_embed = F.embedding(_batch_x, _input_embed)
_bow_embed = _window_embed.sum(dim=1)
_logits = F.linear(_bow_embed, _output_embed)
_logprobs = F.log_softmax(_logits, dim=-1)
_loss = -_logprobs[torch.arange(len(_batch_y)), _batch_y].mean()
_loss.backward()

assert np.isclose(loss, _loss.item())
assert np.allclose(_input_embed.grad.numpy(), input_embed_grad)
assert np.allclose(_output_embed.grad.numpy(), output_embed_grad)

Training loop

In [0]:
running_loss = []
for i in range(0, len(train_x), BS):
    batch_x = train_x[i:i+BS]
    batch_y = train_y[i:i+BS]

    loss, input_embed_grad, output_embed_grad = forward_and_backward(batch_x, batch_y, input_embed, output_embed)
    running_loss.append(loss)

    # gradient updata
    current_lr = LR * 0.995 ** (step // 100)
    input_embed -= current_lr * input_embed_grad
    output_embed -= current_lr * output_embed_grad

    if step % 100 == 0:
        print(f'step = {step}, LR = {current_lr:.3f}, running loss = {np.mean(running_loss):.3f} +- {np.std(running_loss):.3f}')
        running_loss = []

    step += 1

    if step > 5000:
        break

# Inspect Trained Embeddings

Now we can inspect the embedding space. First, we combine input and output embeddings.

In [0]:
final_embed = (input_embed + output_embed) / 2

In [0]:
our_embedding = Embeddings(final_embed, vocab.id2word)

In [0]:
word = 'obama' #@param {type: "string"}
word_embedding = our_embedding.vector(word)
our_embedding.nearest_neighbor(word_embedding, topk=10)

In [0]:
w1 = 'king' #@param {type: "string"}
w2 = 'men' #@param {type: "string"}
w3 = 'women' #@param {type: "string"}
our_embedding.analogy(w1, w2, w3, topk=10)

In [0]:
writer = SummaryWriter('runs/diy')
writer.add_embedding(final_embed, vocab.id2word)
writer.close()

In [0]:
%load_ext tensorboard
%tensorboard --logdir runs/diy

Spoiler Alert: solution to the gradient.

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---



\begin{align}
\frac{\partial}{\partial u_{w_c}} - \log p(w_c|v_{bow})
&= \frac{\partial}{\partial u_{w_c}} - \log \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})} \\
&= \frac{\partial}{\partial u_{w_c}} - \log  \exp(v_{bow} \cdot u_{w_c}) + \log\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - \frac{\partial}{\partial u_{w_c}} v_{bow} \cdot u_{w_c} + \frac{\partial}{\partial u_{w_c}} \log\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - v_{bow} +  \frac {1} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial u_{w_c}} \sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - v_{bow} +  \frac {1} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial u_{w_c}} \exp(v_{bow} \cdot u_{w_c}) \\
&= - v_{bow} +  \frac {\exp(v_{bow} \cdot u_{w_c}) } {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial u_{w_c}} v_{bow} \cdot u_{w_c} \\
&= - v_{bow} +  p(w_c|v_{bow}) v_{bow} \\
\end{align}


\begin{align}
\frac{\partial}{\partial u_w} - \log p(w_c|v_{bow})
&= \frac{\partial}{\partial u_w} - \log \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})} \\
&= \frac{\partial}{\partial u_w} - \log  \exp(v_{bow} \cdot u_{w_c}) + \log\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - \frac{\partial}{\partial u_w} v_{bow} \cdot u_{w_c} + \frac{\partial}{\partial u_w} \log\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= \frac {1} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial u_w} \sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= \frac {1} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial u_w} \exp(v_{bow} \cdot u_w) \\
&= \frac {\exp(v_{bow} \cdot u_w) } {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial u_w} v_{bow} \cdot u_w \\
&= p(w|v_{bow}) v_{bow} \\
\end{align}

Recall $v_{bow} = v_{w_{-2}} + v_{w_{-1}}+ v_{w_{1}}+ v_{w_{2}}$. For each $v \in \{v_{w_{-2}}, v_{w_{-1}}, v_{w_{1}}, v_{w_{2}}\}$
\begin{align}
\frac{\partial}{\partial v} - \log p(w_c|v_{bow})
&= \frac{\partial}{\partial v} - \log \frac {\exp(v_{bow} \cdot u_{w_c})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})} \\
&= \frac{\partial}{\partial v} - v_{bow} \cdot u_{w_c} + \log\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - \frac{\partial}{\partial v} v_{bow} \cdot u_{w_c} + \frac{\partial}{\partial v} \log\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - u_{w_c} + \frac {1} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})}  \frac{\partial}{\partial v} \sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j}) \\
&= - u_{w_c} + \sum_{w_j\in\mathcal{V}} \frac {1} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})} \frac{\partial}{\partial v} \exp(v_{bow} \cdot u_{w_j}) \\
&= - u_{w_c} + \sum_{w_j\in\mathcal{V}} \frac {\exp(v_{bow} \cdot u_{w_j})} {\sum_{w_j\in\mathcal{V}} \exp(v_{bow} \cdot u_{w_j})} \frac{\partial}{\partial v} v_{bow} \cdot u_{w_j}  \\
&= - u_{w_c} + \sum_{w_j\in\mathcal{V}} p(w_j|v_{bow}) u_{w_j} \\
\end{align}
